In [433]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision import models
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm

In [462]:
# 하이퍼파라미터 설정
batch_size = 32
epochs = 150
learning_rate = 0.0001
num_classes = 7  # 얼굴 클래스 개수
val_split = 0.2  # 검증 데이터 비율
early_stop_patience = 3  # 조기 종료 기준 (연속으로 개선되지 않은 epoch 수)

In [463]:
transform = transforms.Compose([
    transforms.RandomRotation(degrees= 10),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p = 0.5),
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.1),
    transforms.GaussianBlur(kernel_size=(5,5)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [464]:
# 데이터셋 로드 (훈련 데이터만 사용)
dataset = datasets.ImageFolder(root='훈련데이터 경로/train_face_bts', transform=transform)
train_size = int((1 - val_split) * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [465]:
print(f"클래스 목록 : {dataset.classes}")

클래스 목록 : ['jhope', 'jimin', 'jin', 'jungkook', 'rm', 'suga', 'v']


In [466]:
# ResNet 모델 불러오기 및 수정
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)  # 마지막 층 수정
model = model.cuda() if torch.cuda.is_available() else model

In [467]:
# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [468]:
# 학습 및 검증 루프
def train_and_evaluate(model, train_loader, val_loader, criterion, optimizer, epochs, early_stop_patience):
    best_val_acc = 0.0
    no_improve_epochs = 0
    
    for epoch in range(epochs):
        # 학습 단계
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        train_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs} - Training')
        
        for images, labels in train_bar:
            images, labels = images.cuda(), labels.cuda()
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            train_bar.set_postfix(loss=running_loss / (train_bar.n + 1), acc=100 * correct / total)
        
        avg_train_loss = running_loss / len(train_loader)
        train_acc = 100 * correct / total
        print(f'Epoch {epoch+1}, Loss: {avg_train_loss:.4f}, Train Accuracy: {train_acc:.2f}%')
        
        # 검증 단계
        model.eval()
        correct, total = 0, 0
        val_bar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{epochs} - Validation')
        
        with torch.no_grad():
            for images, labels in val_bar:
                images, labels = images.cuda(), labels.cuda()
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                val_bar.set_postfix(acc=100 * correct / total)
        
        val_acc = 100 * correct / total
        print(f'Validation Accuracy: {val_acc:.2f}%')
        
        # 조기 종료 조건 확인
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            no_improve_epochs = 0
            torch.save(model.state_dict(), 'resnet_best.pth')  # 최적 모델 저장
        else:
            no_improve_epochs += 1
            if no_improve_epochs >= early_stop_patience:
                print(f'Early stopping at epoch {epoch+1}. No improvement for {early_stop_patience} epochs.')
                break

In [469]:
# 실행
train_and_evaluate(model, train_loader, val_loader, criterion, optimizer, epochs, early_stop_patience)

Epoch 1/150 - Training: 100%|█████████████████████████████████████| 51/51 [00:12<00:00,  3.93it/s, acc=24.4, loss=1.87]


Epoch 1, Loss: 1.8678, Train Accuracy: 24.42%


Epoch 1/150 - Validation: 100%|████████████████████████████████████████████████| 6/6 [00:01<00:00,  5.69it/s, acc=31.9]


Validation Accuracy: 31.87%


Epoch 2/150 - Training: 100%|█████████████████████████████████████| 51/51 [00:12<00:00,  4.08it/s, acc=43.2, loss=1.52]


...

Validation Accuracy: 69.23%


Epoch 15/150 - Training: 100%|███████████████████████████████████| 51/51 [00:12<00:00,  4.07it/s, acc=97.2, loss=0.113]


Epoch 15, Loss: 0.1127, Train Accuracy: 97.18%


Epoch 15/150 - Validation: 100%|███████████████████████████████████████████████| 6/6 [00:01<00:00,  5.88it/s, acc=64.8]

Validation Accuracy: 64.84%
Early stopping at epoch 15. No improvement for 3 epochs.


In [397]:
# 최종 모델 저장
torch.save(model.state_dict(), 'resnet_final7.pth')

In [470]:
# 최종 모델 저장
torch.save(model.state_dict(), 'resnet_bts1.pth')